In [2]:
import pyranges as pr
import seaborn as sns
import yaml
import os
import sys
import cerberus
# from snakemake.io import expand

p = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))+'/scripts/'
print(p)
sys.path.append(p)

from sm_utils import *


/Users/fairliereese/Documents/programming/mele_lab/projects/240706_pantranscriptome_cerberus_gtf_merge/scripts/


In [3]:
config = load_config()
od = '../../../'

In [15]:
analysis = 'espresso_pseudomasked_genomic'
tool = 'espresso'
config_tsv = od+f'snakemake/config_{analysis}_expression.tsv'
meta_df = parse_config(config_tsv)
meta_df['analysis'] = analysis
tech_rep = ['GM10493_1']

In [16]:
import pyranges as pr
import pandas as pd

In [17]:
def make_hier_entry(df, how='t'):
    """
    kind {'g','t'}
    """
    agg_dict = {'min_coord': 'min', 'max_coord': 'max'}
    t_df = df.copy(deep=True)
    t_df['min_coord'] = t_df[['Start', 'End']].min(axis=1)
    t_df['max_coord'] = t_df[['Start', 'End']].max(axis=1)
    if how == 't':
        gb_cols = ['Chromosome', 'Strand', 'gene_name',
                   'gene_id', 'transcript_id', 'transcript_name',
                   'tss_id', 'tes_id',
                   'new_transcript_id', 'original_transcript_id',
                   'original_transcript_name', 'ag1', 'ag2']
    elif how == 'g':
        gb_cols = ['Chromosome', 'Strand', 'gene_name',
                   'gene_id']
    gb_cols = list(set(gb_cols)&(set(t_df.columns)))

    cols = gb_cols + ['min_coord', 'max_coord']
    t_df = t_df[cols]
    t_df = t_df.groupby(gb_cols, observed=True).agg(agg_dict).reset_index()
    t_df.rename({'min_coord': 'Start', 'max_coord': 'End'}, axis=1, inplace=True)
    if how == 't':
        t_df['Feature'] = 'transcript'
    elif how == 'g':
        t_df['Feature'] = 'gene'

    return t_df

In [18]:
# files = expand(od+config['fmt']['novel_gene_rename_gtf'], 
#        tech_rep=tech_rep,
#        analysis=analysis)
# for f in files:
f = '../../../data/espresso_pseudomasked_genomic/fmt/GM10493_1_novel_gene_rename.gtf'
df = pr.read_gtf(f).df

In [19]:
beep = df.loc[df.gene_id.str.contains('novel')].gene_id.unique().tolist()
print(len(beep))

1360


In [20]:
# if tool == 'iq':
#     # just grab all the gene ids to be the gene names
#     df['gene_name'] = df['gene_id']

#     # for transcripts / other transcript-level features do the same but restrict to those feats
#     inds = df.loc[df.Feature != 'gene'].index
#     df.loc[inds, 'transcript_name'] = df.loc[inds, 'transcript_id']

In [21]:
# make gene entries

print(len(df.index))
print(len(df.gene_id.unique()))
g_df = make_hier_entry(df, how='g')
print(len(g_df.loc[g_df.Feature=='gene'].index))

215498
14336
14336


In [22]:
beep = g_df.loc[g_df.gene_id.str.contains('novel')].gene_id.unique().tolist()
print(len(beep))

1360


In [23]:
g_df.loc[g_df.gene_id.duplicated(keep=False)].sort_values(by='gene_id')

,Chromosome,gene_id,Strand,Start,End,Feature


In [24]:
# concat them and then sort gtf
df = pd.concat([df, g_df], axis=0)
df = cerberus.sort_gtf(df)
df = pr.PyRanges(df)

df.to_gtf('test.gtf')